In [6]:
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "..."}]}

{'messages': [{'role': 'system', 'content': 'You are helpful'},
  {'role': 'user', 'content': "What's the capital of France?"},
  {'role': 'assistant', 'content': '...'}]}

In [7]:
""" json.dumps(): 将 Python 对象编码成 JSON 字符串。 json.dump(): 将 Python 对象编码成 JSON 字符串，并写入到文件中。json.loads(): 将已编码的 JSON 字符串解码为 Python 对象。json.load(): 读取文件中的 JSON 编码的数据，并解码为 Python 对象。
    """
import os
import json
import torch
torch.cuda.is_available()
def json_standard(input_object):

    # print(input_object,type(input_object))
    return json.dumps(input_object,ensure_ascii=False,indent=4)
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def check_file(file_path:str):
    """返回该路径下是否存在文件"""
    is_exist=os.path.exists(file_path)
    if is_exist:
        file_name=file_path.split("/")[-1]
        print(f"{file_name} exist")
    return is_exist

import torch
import json

from transformers import (AutoTokenizer)
model_id="/home/ckqsudo/code2024/0model/Mistral-7B-Instruct-v0.2"
tokenizer=AutoTokenizer.from_pretrained(model_id)
# 这里不设置 trust_remote_code也行
tokenizer.padding_size='right'#填充符的位置


In [8]:
# tokenizer("你好啊")

In [9]:
en_data=read_file("/home/ckqsudo/code2024/Mistral-7b-finetune/data/en_regular_cache5.json")

In [10]:
def regular_data(cur_data):
    for x,item in enumerate(cur_data):
        for y,dialog_item in enumerate(item["topic_dialog"]["dialog"]):
            text=dialog_item["text"]
            while text.startswith(" "):
                text=text[1:]
            while text.endswith(" "):
                text=text[:-1]
            cur_data[x]["topic_dialog"]["dialog"][y]["text"]=text
regular_data(en_data)

In [11]:
len(en_data)

520

In [12]:
en_data[0]

{'theme': 'cognitive behaviour therapy: exploring cognition through images and thoughts in therapy to uncover emotionally charged thoughts and beliefs.',
 'is_same_qa': 0,
 'is_same_case': 0,
 'background': 'A client experiences anxiety while walking down the high street due to negative self-perceptions.',
 'pre_reasoning': 'The term ‘cognition’ includes images as well as thoughts. Some clients might find it difficult to pinpoint their NATs in a specific situation, but the therapist can ask such clients if they had any pictures or images in their minds in that situation which might be more retrievable. Using imagery can help clients to relive past situations in order to uncover the hot (emotionally charged) thoughts associated with these situations.',
 'topic_dialog': {'key': 'fewshot',
  'unit_id': 0,
  'method': 'Cognitive Behaviour Therapy',
  'dialog': [{'id': '0-0',
    'participant': 'Client',
    'text': 'I’m walking down the high street. I feel very uncomfortable. I think every

In [13]:
from copy import deepcopy

In [53]:
# from ast import Raise
import datasets
import copy
def dialog_item_str(dialog_item:dict):
    """将dialog_item 转为str"""
    return dialog_item["participant"]+": "+dialog_item["text"]+""

def msg_analysis_annotation(msg_item:dict,task_define=str,task_define_dict={"dialog":"Please generate the response of the dialog.","reasoning":"Please analysis the reasoning of this response.","observation":"Please summarize the observations of this response."}):
    # print(msg_item)
    if "annotation" not in msg_item.keys() or task_define not in task_define_dict.keys() and task_define_dict!="dialog":
        print("!!!!!!!!!!!!!!!!!!!!",msg_item.keys())
        
        raise ValueError("")
    # change system prompt
    msg_item["messages"][0]['content'] = msg_item["messages"][0]['content'].replace(task_define_dict['dialog'], task_define_dict[task_define])
    text_observation=''
    for idx,item in enumerate(msg_item["messages"]):
        if idx!=0:
            if task_define=="observation" and 'user' in msg_item["messages"][idx]['role']:
                text_observation=msg_item["annotation"][idx]['content']+ " "
            else:
                if 'assistant' in msg_item["messages"][idx]['role']:
                    # prev user=therapis
                    msg_item["messages"][idx-1]['content']+=" "+msg_item["messages"][idx]['content']
                    
                    if task_define=="reasoning":
                        # prev user=therapist
                        # 将对话记录不计算loss
                        text_reasoning= msg_item["annotation"][idx]['content'] + " "
                        annotation_text=text_reasoning
                        text_reasoning=''
                    elif  task_define=="observation":
                        #prev user=therapist
                        annotation_text=text_observation
                        text_observation=''

                    msg_item["messages"][idx]['content']= annotation_text
                    # change target
    return msg_item

def msg_add_annotation(msg_item:dict,add_observation=False):
    # print(msg_item)
    if "annotation" not in msg_item.keys():
        print("!!!!!!!!!!!!!!!!!!!!",msg_item.keys())
        
        raise ValueError("")
    prefix_observation=""
    for idx,item in enumerate(msg_item["messages"]):
        if idx!=0:
            if add_observation==True and 'user' in msg_item["messages"][idx]['role']:
                prefix_observation="Observation"+": "+msg_item["annotation"][idx]['content']+ "\n"+" "
            else:
                if 'assistant' in msg_item["messages"][idx]['role']:
                    text_reasoning="Reasoning" + ": " + msg_item["annotation"][idx]['content'] + "\n"+" "
                    text_with_annotation = prefix_observation + text_reasoning + msg_item["messages"][idx]['content']
                    msg_item["messages"][idx]['content']=text_with_annotation
    return msg_item
    
def get_token_len(py_obj:dict,idx=None):
    len_counter=0
    if "annotation" not in py_obj.keys():
        raise ValueError()
    if idx==None:
        idx=len(py_obj["messages"])
    len_counter+=len(tokenizer(str(py_obj["messages"][:idx]))['input_ids'])
    len_counter+=len(tokenizer(str(py_obj["annotation"][:idx]))['input_ids'])
    return len_counter
def check_msg_format(msg_item):
    role_list=[item["role"] for item in msg_item["messages"]]
    if len(set(role_list))==1:
        return False
    return True

def chat_assistant_format(msg_item):
    # list 中必须要有一个"assistant"，例子中必须要是 user assistant user的形式
    
    new_annotation_list,new_msg_list=[],[]
    for idx,item in enumerate(msg_item["messages"]):
        if len(new_msg_list)!=0 and new_msg_list[-1]["role"]==msg_item["messages"][idx]["role"]: # 如果存在重复项
            if new_msg_list[-1]["content"].startswith("Therapist:") and msg_item["messages"][idx]["content"].startswith("Therapist:"):
                # Therapist拼接
                # print(new_msg_list[-1]["content"],item["content"])
                new_msg_list[-1]["content"]+=" "+item["content"][len("Therapist:"):]
                new_annotation_list[-1]["content"]+=" "+msg_item["annotation"][idx]["content"]
            else:
                # Client 拼接
                new_msg_list[-1]["content"]+=" "+item["content"]
                new_annotation_list[-1]["content"]+=" "+msg_item["annotation"][idx]["content"]
            # 如果相等则拼接 一定要确保各异
        else:
            new_msg_list.append(msg_item["messages"][idx])
            new_annotation_list.append(msg_item["annotation"][idx])
    msg_item["messages"]=new_msg_list
    msg_item["annotation"]=new_annotation_list
    return msg_item
def limit_incontext_windows(max_seq_len,tokenizer,msg_item,margin=100):
    if max_seq_len<1000:
        raise ValueError("")
    # 将返回一个限制长度的messages 和一个没有限制长度的messages
    idx=len(msg_item["messages"])
    while get_token_len(msg_item,idx)>max_seq_len-margin:
        idx=idx-1
    if len(msg_item["annotation"][idx:])==1 and len(msg_item["annotation"][:idx])>9:
        idx-=4
    old_msg_item={
        "messages": msg_item["messages"][0:idx],
        "annotation": msg_item["annotation"][0:idx],
        "background":msg_item["background"]
    }
    # "annotation":[{"role":"pre_observation","content":""}]
    new_msg_item={
        "messages": msg_item["messages"][idx:],
        "annotation": msg_item["annotation"][idx:],
        "background":msg_item["background"]
    }# therapist 开头的
    dialog_history_str=" ".join([item["role"]+":"+item["content"] for item in old_msg_item["messages"][-3:]])
    new_msg_item["annotation"].insert(0,{"role":"pre_observation","content":""})
    new_msg_item["messages"].insert(0,
                                    {"role": "user", "content": """You are a friendly and professional AI psychotherapist who always provides useful emotional support. There is a real psychological counseling case. There is a background: """+msg_item['background']+""" Please generate the response of the dialog. There is the dialog history: """+dialog_history_str})
    print("切分",len(old_msg_item["messages"]),len(new_msg_item["messages"]))
    return old_msg_item,new_msg_item

def mistral_data_preprocess(idx,topic_dialog,prefix_unit):
    # User
    system_prompt="""You are a friendly and professional AI psychotherapist who always provides useful emotional support. There is a real psychological counseling case. There is a background: """+topic_dialog['background']+""" Please generate the response of the dialog. """
    ## 如果开头为Therapist 就用对话历史
    if "Therapist" in topic_dialog["topic_dialog"]["dialog"][0]["participant"] and idx!=0:
        if topic_dialog["is_same_qa"]==1 or topic_dialog["is_same_case"]==1:
            system_prompt+="There is the dialog history: "
            for dialog_item in prefix_unit["topic_dialog"]["dialog"][-3:]:
                system_prompt+=dialog_item_str(dialog_item)+" "
    tmp_result={"messages": [
    {"role": "user", "content": system_prompt},
    ],          "annotation":[
    {"role":"pre_observation","content":""}],
    "background":topic_dialog['background']}
    
    for idx,dialog_item in enumerate(topic_dialog["topic_dialog"]["dialog"]):
        if "Therapist" in dialog_item["participant"]:
            # print(dialog_item.keys())
            tmp_result["messages"].append({"role": "assistant", "content": dialog_item_str(dialog_item)})
            tmp_result["annotation"].append({"role": "reasoning", "content": dialog_item['reasoning']})
        else:
            tmp_result["messages"].append({"role": "user", "content": dialog_item_str(dialog_item)})
            tmp_result["annotation"].append({"role": "observation", "content": dialog_item["observation"]})
    """注意！：最后一定使用函数使得集合符合User/assistant/user/assistant的形式"""
    return tmp_result

def data_preprocess_main(cur_data,tokenizer,max_seq_len):
    msg_list=[]
    for idx,item in enumerate(cur_data[:]):
        msg_item=mistral_data_preprocess(idx=idx,topic_dialog=item,prefix_unit=cur_data[idx-1])
        msg_item=chat_assistant_format(msg_item)
        msg_list.append(msg_item)
    result_msg_list=[]
    for idx,msg_item in enumerate(msg_list):
        counter=0
        while get_token_len(msg_item)>max_seq_len:
            print(idx,counter)
            counter+=1
            old_msg_item,new_msg_item=limit_incontext_windows(max_seq_len,tokenizer,msg_item)
            old_msg_item=chat_assistant_format(old_msg_item)
            result_msg_list.append(old_msg_item)
            msg_item=new_msg_item.copy()
        msg_item=chat_assistant_format(msg_item)
        result_msg_list.append(msg_item)
    print("切分完成",len(result_msg_list))
    for idx,item in enumerate(result_msg_list):
        result_msg_list[idx]=chat_assistant_format(result_msg_list[idx])
        if check_msg_format(result_msg_list[idx])==False:
            print(idx)
            raise ValueError("")
    print("格式拼接完成",len(result_msg_list))

    dialog_format_list=[pure_dialog, merge_r_dialog, merge_o_r_dialog, only_r_task, only_o_task] = [copy.deepcopy(result_msg_list) for _ in range(5)]
    # 列表是浅拷贝
    print("拷贝完成",len(pure_dialog))
    for idx,_ in enumerate(pure_dialog):
        merge_r_dialog[idx]=msg_add_annotation(merge_r_dialog[idx],add_observation=False)
        merge_o_r_dialog[idx]=msg_add_annotation(merge_o_r_dialog[idx],add_observation=True)
        only_r_task[idx]=msg_analysis_annotation(only_r_task[idx],task_define="reasoning")
        only_o_task[idx]=msg_analysis_annotation(only_o_task[idx],task_define="observation")
        for x,_ in enumerate(dialog_format_list):
            if "background" in dialog_format_list[x][idx].keys():
                del dialog_format_list[x][idx]["background"]# sb 的 GPT
            if "annotation" in dialog_format_list[x][idx].keys():
                del dialog_format_list[x][idx]["annotation"]# sb 的 GPT
    print("",len(pure_dialog))
    return dialog_format_list
# def regular_data()
pure_dialog,annote_dialog,annote2_dialog,only_r_dialog,only_o_dialog=data_preprocess_main(en_data,tokenizer,max_seq_len=4096-300)

71 0
切分 49 15
72 0
切分 43 9
75 0
切分 38 11
90 0
切分 35 19
98 0
切分 50 16
99 0
切分 55 27
100 0
切分 51 145
100 1
切分 51 95
100 2
切分 59 37
326 0
切分 34 11
436 0
切分 47 27
449 0
切分 53 23
470 0
切分 50 23
479 0
切分 46 67
479 1
切分 47 21
495 0
切分 42 12
504 0
切分 49 24
切分完成 537
格式拼接完成 537
拷贝完成 537
 537


In [54]:
show_idx=45
pure_dialog[show_idx]['messages'][3:5],annote_dialog[show_idx]['messages'][3:5],annote2_dialog[show_idx]['messages'][3:5]

([{'role': 'assistant',
   'content': 'Therapist: Encouraging that you are both motivated. Focus on maintaining motivation and identifying moments of higher optimism.'}],
 [{'role': 'assistant',
   'content': "Reasoning: The therapist acknowledges the couple's shared motivation, normalizes their feelings of low optimism, and encourages them to focus on sustaining their motivation while recognizing instances of increased optimism for progress in therapy.\n Therapist: Encouraging that you are both motivated. Focus on maintaining motivation and identifying moments of higher optimism."}],
 [{'role': 'assistant',
   'content': "Observation: Dick expresses his high motivation to work on the marriage but acknowledges a lack of optimism, indicating a realistic self-assessment of their current situation. Karen echoes Dick's sentiment of being motivated yet lacking optimism, showing a shared perspective on their marital challenges and the need for improvement.\n Reasoning: The therapist acknowle

In [55]:
only_r_dialog[show_idx]['messages'][2:5],only_o_dialog[show_idx]['messages'][2:5]

([{'role': 'user',
   'content': 'Dick: I am highly motivated to save our marriage but lack optimism. I rate us low in optimism for improving our relationship. Karen: I am also highly motivated but lack optimism. Our scores align closely. Therapist: Encouraging that you are both motivated. Focus on maintaining motivation and identifying moments of higher optimism.'},
  {'role': 'assistant',
   'content': "The therapist acknowledges the couple's shared motivation, normalizes their feelings of low optimism, and encourages them to focus on sustaining their motivation while recognizing instances of increased optimism for progress in therapy. "}],
 [{'role': 'user',
   'content': 'Dick: I am highly motivated to save our marriage but lack optimism. I rate us low in optimism for improving our relationship. Karen: I am also highly motivated but lack optimism. Our scores align closely. Therapist: Encouraging that you are both motivated. Focus on maintaining motivation and identifying moments of

In [56]:
len(pure_dialog),len(annote_dialog),len(annote2_dialog)

(537, 537, 537)

In [57]:
from datasets import Dataset
# chat_pure_dialog,annote_dialog

pure_dialog_64= Dataset.from_list(pure_dialog)
annotate_64= Dataset.from_list(annote_dialog)
annotate_65_2= Dataset.from_list(annote2_dialog)
only_r_67=Dataset.from_list(only_r_dialog)
only_o_67=Dataset.from_list(only_o_dialog)
len(annotate_64['messages']),len(pure_dialog_64['messages']),len(annotate_65_2['messages'])

(537, 537, 537)

In [58]:
check_list=[]
for x,item in enumerate(only_r_67["messages"]):
    check_str=""
    for idx,dialog_item in enumerate(item):
        check_str+=dialog_item["role"]
        # print(tokenizer.apply_chat_template(item))
    if check_str.startswith('user')==False:
        print(check_str)
    if 'useruser' in check_str:
        print(idx,x,check_str)
    if 'assistantassistant' in check_str:
        print(check_str)
    if 'assistant' not in check_str:
        print('assistant',x,check_str)
    if 'user' not in check_str:
        print('user',x,check_str)

In [59]:
len(annotate_64["messages"])

537

In [61]:
prefix="/home/ckqsudo/code2024/Mistral-7b-finetune/hf"
# pure_dialog_64.save_to_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_pure_dialog_dataset_3")
# annotate_64.save_to_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_reasoning_dataset_3")
# annotate_65_2.save_to_disk("/home/ckqsudo/code2024/Mistral-7b-finetune/hf_en_observation_reasoning_dataset_3")
only_o_67.save_to_disk(f"{prefix}_en_pure_observation_dataset_3")
only_r_67.save_to_disk(f"{prefix}_en_pure_reasoning_dataset_3")

Saving the dataset (0/1 shards):   0%|          | 0/537 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/537 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/537 [00:00<?, ? examples/s]

In [13]:


# re=[]
# for item in chat_format_dialog["text"]:
#     re.extend([list(d_i.keys()) for d_i in item["messages"]])
# a=[]
# for r in re:
#     a.extend(r)
# set(a)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1031644